In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm
import re
import time

In [2]:
list_topic=['politics','money','culture','society','world']
list_subtopic=[['assemgov','bluehouse','diplomacy','nk','northkorea'],
               ['indusrty','finance','science','globaleconomy','labor'],
               ['book','perfomance','art','classic','song','broadcast','movie'],
              ['law','education','accident','welfare','traffic','environment','region','health'],
              ['northame','china','japan','eu','etc']]
topic_dict={'politics':"정치",'money':"경제",'culture':"라이프",'society':"사회",'world':"국제"}

In [3]:
import pandas as pd
import numpy as np
#기간설정
dt_index = pd.date_range(start='20200101', end='20200103')

In [4]:
str(dt_index.date[0])

'2020-01-01'

In [5]:
# 각각의 리스트 생성
topic_list=[]
date_list=[]
title_list=[]
text_list=[]

In [6]:
def no_space(text):
    text1=re.sub('&nbsp; | &nbsp;| \n|\t|\r','',text)
    text2=re.sub('\n\n','',text1)
    return text2

In [7]:
#크롤링 def
def get_news(date,main_url,sub_url,topic,sub_topic):
    html = urlopen(main_url+date)
    bsObject = BeautifulSoup(html, "html.parser")
    print(bsObject)
    
    for meta in bsObject.select('#content span.thumb a'):
        link=meta['href']
        date=date
        url=urlopen(sub_url+link)
        bs_obj=BeautifulSoup(url, "html.parser")
        title=bs_obj.select_one("title").text.split('-')[0]
        x=bs_obj.select_one("#article_body").text
        title=no_space(title)
        text=no_space(x)
        date_list.append(date)
        topic_list.append(topic_dict[topic])
        title_list.append(title)
        text_list.append(text)

    if bsObject.select('#content a.link_page')!=[]:
        #재귀 호출
        url_new="https://news.joins.com/"+topic+"/"+sub_topic+"list/2?Date="
        return get_news(date,url_new,sub_url,topic,sub_topic)
    else:
        return bsObject

In [8]:
#크롤링 시작
num=0
for num_main in tqdm(range(len(list_topic)),desc='Topic_Progress',leave=False):
    for num_sub in tqdm(range(len(list_subtopic[num])),desc='Sub_Topic_Progress',leave=False):
        main_url="https://news.joins.com/"+list_topic[num_main]+"/"+list_subtopic[num_main][num_sub]+"/list/1?filter=OnlyJoongang&date="
        sub_url='https://news.joins.com/article/'
        for num_date in tqdm(range(len(dt_index.date)),desc='Day_Progress'):
            date=str(dt_index.date[num_date])
            bs=get_news(date,main_url,sub_url,list_topic[num_main],list_subtopic[num][num_sub])
            time.sleep(0.01)
            
            
    num+=1



<!DOCTYPE html>

<!--[if IE]><![endif]-->
<!--[if lt IE 7 ]><html class="no-js ie6" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraph.org/schema/" lang="ko" xml:lang="ko"><![endif]-->
<!--[if IE 7 ]><html class="no-js ie7" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraph.org/schema/" lang="ko" xml:lang="ko"><![endif]-->
<!--[if IE 8 ]><html class="no-js ie8" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraph.org/schema/" lang="ko" xml:lang="ko"><![endif]-->
<!--[if IE 9 ]><html class="no-js ie9" xmlns:fb="http://ogp.me/ns/fb#" lang="ko" xml:lang="ko"><![endif]-->
<!--[if (gt IE 9)|!(IE)]><!-->
<html class="no-js" lang="ko" xml:lang="ko" xmlns:fb="http://ogp.me/ns/fb#">
<!--<![endif]-->
<head>
<title>정치 &gt; 국회정당 뉴스 - 중앙일보</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width" name="viewport"/>
<meta content="IE=edge" http-equiv="X-UA-Compat


<!DOCTYPE html>

<!--[if IE]><![endif]-->
<!--[if lt IE 7 ]><html class="no-js ie6" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraph.org/schema/" lang="ko" xml:lang="ko"><![endif]-->
<!--[if IE 7 ]><html class="no-js ie7" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraph.org/schema/" lang="ko" xml:lang="ko"><![endif]-->
<!--[if IE 8 ]><html class="no-js ie8" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraph.org/schema/" lang="ko" xml:lang="ko"><![endif]-->
<!--[if IE 9 ]><html class="no-js ie9" xmlns:fb="http://ogp.me/ns/fb#" lang="ko" xml:lang="ko"><![endif]-->
<!--[if (gt IE 9)|!(IE)]><!-->
<html class="no-js" lang="ko" xml:lang="ko" xmlns:fb="http://ogp.me/ns/fb#">
<!--<![endif]-->
<head>
<title>정치 &gt; 국회정당 뉴스 - 중앙일보</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width" name="viewport"/>
<meta content="IE=edge" http-equiv="X-UA-Compat

KeyboardInterrupt: 

In [10]:
#판다스로 저장 (후에 csv파일로 저장 라인 추가)

df=pd.DataFrame({"date":date_list, "title":title_list, "topic":topic_list,"text":text_list}) 
print(df) 

Empty DataFrame
Columns: [date, title, topic, text]
Index: []
